In [ ]:
import tweepy
import networkx

In [ ]:
europython_id = 15324940

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
results = []
max_id = None
while True:
    try:
        print("do a request, currently {} results".format(len(results)))
        statuses = api.search('@europython since:2016-07-17 until:2016-07-22',
                              count=100,
                              max_id=max_id)
        if statuses:
            results.extend(statuses)
            max_id = min([x.id for x in statuses]) - 1
        else:
            print("done with %d results" % len(results))
            break
    except KeyboardInterrupt:
        print('stopped by user')
        break

In [ ]:
network = networkx.Graph()
for status in results:
    if status.user.id not in network.node:
        network.add_node(status.user.screen_name, name=status.user.name)
    
    for user in status.entities['user_mentions']:
        if user['id'] not in network.node:
            network.add_node(user['screen_name'], name=user['name'])
        try:
            network.edge[status.user.screen_name][user['screen_name']]['weight'] += 1
        except KeyError:
            network.add_edge(status.user.screen_name, user['screen_name'], weight=1)

In [ ]:
network.number_of_nodes()

In [ ]:
networkx.write_gml(network, "ep2016-2.gml")